In [1]:
import pandas as pd 
import folium
import branca.colormap as cm
import numpy as np 
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()
from shapely.geometry import LineString, Point
import pickle
import osmnx as ox

In [2]:
# バス停と最寄りノードのcsvファイルを読み込む
osmid_df = pd.read_csv('data/osmid.csv')

# 数値列に変換
osmid_df['osmid'] = osmid_df['osmid'].astype(int)


In [3]:
# 全体のosmidのpath
osmnx_path = 'data/osmnx_graph.pkl'
G = pickle.load(open(osmnx_path, "rb"))
G = ox.project_graph(G)
# 元のMultiDiGraphをgpd.GeoDataFrameに変換
nodes, edges = ox.graph_to_gdfs(G)
nodes = nodes.reset_index()

In [4]:
# 全体のosmidのpath
osmnx_path = 'osmnx_graph.pkl'
all_G = pickle.load(open(osmnx_path, "rb"))
all_G = ox.project_graph(all_G)
# 元のMultiDiGraphをgpd.GeoDataFrameに変換
# nodes, edges = ox.graph_to_gdfs(all_G)
# nodes = nodes.reset_index()

In [5]:
# 座標を付与
osmid_df = pd.merge(osmid_df,nodes,on=['osmid'],how='left')

In [6]:
# stop_nameを付与する
stop_name = pd.read_csv('data/GTFS/stops.txt')
stop_name = stop_name[['stop_id','stop_name']]
osmid_df = pd.merge(osmid_df,stop_name,on=['stop_id'])

In [7]:
# drive（drive netが欠損なもの ）
drive_osmid_df = osmid_df.loc[~osmid_df['y'].isna()].reset_index(drop=True)

all_osmid_df = osmid_df.loc[osmid_df['y'].isna()].reset_index(drop=True)

In [8]:
route_segment_df = pd.read_csv('data/bus_stop_segment.csv')

In [9]:
# merge1
merge_df = pd.merge(osmid_df,route_segment_df,left_on=['stop_id'],right_on=['from_stop_id'],how='right')

# rename
merge_df = merge_df.rename(columns={'osmid':'from_osmid','lon':'from_lon','lat':'from_lat'})

# # 不要なカラムを整理する
merge_df = merge_df[['from_stop_id','to_stop_id','from_stop_name','from_osmid','from_lon','from_lat']]

# # merge1 
merge_df = pd.merge(merge_df,osmid_df,left_on=['to_stop_id'],right_on=['stop_id'],how='left')

# # # rename
merge_df = merge_df.rename(columns={'osmid':'to_osmid','stop_name':'to_stop_name','lon':'to_lon','lat':'to_lat'})

# # # 不要なカラムを整理する
merge_df = merge_df[['from_stop_id','to_stop_id','from_stop_name','to_stop_name','from_osmid','to_osmid','from_lon','from_lat','to_lon','to_lat']]


In [10]:
merge_df

,from_stop_id,to_stop_id,from_stop_name,to_stop_name,from_osmid,to_osmid,from_lon,from_lat,to_lon,to_lat
0,255_1,10_4,新山口駅,常盤駅入口,1346410226,5276614767,131.396014,34.094593,131.290347,33.941936
1,10_4,29_1,常盤駅入口,山口宇部空港,5276614767,1998295607,131.290347,33.941936,NaN,NaN
2,29_1,10_3,山口宇部空港,常盤駅入口,1998295607,5308433224,NaN,NaN,131.291513,33.943339
3,10_3,255_1,常盤駅入口,新山口駅,5308433224,1346410226,131.291513,33.943339,131.396014,34.094593
4,2_2,294_1,宇部新川駅,ＡＮＡクラウンプラザホテル宇部,1148212452,1666722830,131.242537,33.957621,131.244143,33.954945
...,...,...,...,...,...,...,...,...,...,...
687,289_2,223_2,阿武瀬臨時１,阿武瀬,1350962795,1350960984,131.326666,34.129105,131.333953,34.125435
688,223_2,290_2,阿武瀬,西市小野臨時,1350960984,2349420307,131.333953,34.125435,131.338015,34.123789
689,290_2,224_2,西市小野臨時,西市小野,2349420307,1350960877,131.338015,34.123789,131.341101,34.124612
690,224_2,291_2,西市小野,市小野臨時,1350960877,2349420403,131.341101,34.124612,131.342403,34.126308


In [11]:
# 名寄せ（GTFSをバカ日本電気の表記に統一する）
merge_df.loc[merge_df['from_stop_name'].str.contains('宇部興産中央病院', na=False),'from_stop_name'] = '宇部興産中央病院（中）'
merge_df.loc[merge_df['to_stop_name'].str.contains('宇部興産中央病院', na=False),'to_stop_name'] = '宇部興産中央病院（中）'
merge_df.loc[merge_df['from_stop_name'].str.contains('八幡宮', na=False), 'from_stop_name'] = '八幡宮'
merge_df.loc[merge_df['to_stop_name'].str.contains('八幡宮', na=False), 'to_stop_name'] = '八幡宮'

In [12]:
# 欠損値（allnet箇所）のroute_segmentを削除する
drive_df = merge_df.dropna().reset_index(drop=True)

# 欠損値を含む行（all net）のみを取得
all_df = merge_df[merge_df.isna().any(axis=1)].reset_index(drop=True)


In [14]:
# 最短ルート（drive）
drive_df['path'] = drive_df.progress_apply(lambda row: ox.distance.shortest_path(G, row['from_osmid'], row['to_osmid']), axis=1)

  0%|          | 0/644 [00:00<?, ?it/s]/var/folders/dv/f5dwn22136x12k7hhgnq1jjm0000gn/T/ipykernel_56700/1612590031.py:2: FutureWarning: The `shortest_path` function has moved to the `routing` module. Calling it via the `distance` module will raise an error starting in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  drive_df['path'] = drive_df.progress_apply(lambda row: ox.distance.shortest_path(G, row['from_osmid'], row['to_osmid']), axis=1)
100%|██████████| 644/644 [02:01<00:00,  5.31it/s]


In [15]:
# 最短ルート（all）
all_df['path'] = all_df.progress_apply(lambda row: ox.distance.shortest_path(all_G, row['from_osmid'], row['to_osmid']), axis=1)

  0%|          | 0/48 [00:00<?, ?it/s]/var/folders/dv/f5dwn22136x12k7hhgnq1jjm0000gn/T/ipykernel_56700/3792700106.py:2: FutureWarning: The `shortest_path` function has moved to the `routing` module. Calling it via the `distance` module will raise an error starting in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  all_df['path'] = all_df.progress_apply(lambda row: ox.distance.shortest_path(all_G, row['from_osmid'], row['to_osmid']), axis=1)
100%|██████████| 48/48 [00:13<00:00,  3.49it/s]


In [16]:
def compute_shortest_path(G, source, target, via_nodes=None):
    """
    route修正アルゴリズム
    via_nodes : list
    """
    # try:
    if via_nodes and isinstance(via_nodes, list) and len(via_nodes) > 0:
        path = []
        current = source
        for via in via_nodes:
            # ソースから最初の経由ノードへ
            sub_path = ox.shortest_path(G, current, via, weight='length')
            if path:
                # 先頭ノードが重複するため除外
                path += sub_path[1:]
            else:
                path = sub_path
            current = via
        # 最後の経由ノードからターゲットへ
        sub_path = ox.shortest_path(G, current, target, weight='length')
        if path:
            path += sub_path[1:]
        else:
            path = sub_path
    else:
        # 経由ノードがない場合
        path = ox.shortest_path(G, source, target, weight='length')
    return path
    # except ox.NetworkXNoPath:
    #     return None  # パスが存在しない場合
    
route = compute_shortest_path(G, 11848722368, 2334406595, via_nodes=[1351027617,1351027562])

In [62]:
# 各行のosmidリストからLineStringまたはPointを作成する関数
def create_geometry(osmid_list,graph):
    coords = [(graph.nodes[node]['x'], graph.nodes[node]['y']) for node in osmid_list]
    
    if len(coords) > 1:
        return LineString(coords)  # 複数の座標がある場合はLineString
    elif len(coords) == 1:
        return Point(coords[0])  # 座標が1つしかない場合はPoint
    else:
        return None  # 座標がない場合はNone

In [18]:
# 各行のosmidリストからLineStringまたはPointを作成する関数
def all_create_geometry(osmid_list):
    coords = [(all_G.nodes[node]['x'], all_G.nodes[node]['y']) for node in osmid_list]
    
    if len(coords) > 1:
        return LineString(coords)  # 複数の座標がある場合はLineString
    elif len(coords) == 1:
        return Point(coords[0])  # 座標が1つしかない場合はPoint
    else:
        return None  # 座標がない場合はNone

In [70]:
drive_df['geometry'] = drive_df['path'].progress_apply(lambda x: create_geometry(x, G))

# # GeoDataFrameを作成し、CRSを設定
drive_gdf = gpd.GeoDataFrame(drive_df, geometry='geometry', crs=G.graph['crs'])

# # 緯度経度 (EPSG:4326) に変換
drive_gdf = drive_gdf.to_crs(epsg=4326)

100%|██████████| 644/644 [00:00<00:00, 22678.58it/s]


In [71]:
all_df['geometry'] = all_df['path'].progress_apply(lambda x: create_geometry(x, all_G))

# GeoDataFrameを作成し、CRSを設定
all_gdf = gpd.GeoDataFrame(all_df, geometry='geometry', crs=all_G.graph['crs'])

# 緯度経度 (EPSG:4326) に変換
all_gdf = all_gdf.to_crs(epsg=4326)

100%|██████████| 48/48 [00:00<00:00, 22089.82it/s]


In [80]:
drive_compute_shortes_df = pd.read_excel('data/compute_shortest_path.xlsx')

drive_compute_shortes_df = pd.merge(drive_compute_shortes_df,drive_df,how='left',on=['from_stop_name','to_stop_name'])

# # リストに入れるために文字列に変更
drive_compute_shortes_df['via'] = drive_compute_shortes_df['via'].astype('str')

# # via列をリスト形式に変換
drive_compute_shortes_df["via"] = drive_compute_shortes_df["via"].apply(lambda x: list(map(int, x.split(','))))

drive_compute_shortes_df['path'] = drive_compute_shortes_df.progress_apply(lambda row: compute_shortest_path(G, row['from_osmid'],row['to_osmid'], row['via']), axis=1)
# # # 最短ルート
drive_compute_shortes_df['geometry'] = drive_compute_shortes_df['path'].progress_apply(lambda x: create_geometry(x, G))

# # GeoDataFrameを作成し、CRSを設定
drive_compute_shortes_df = gpd.GeoDataFrame(drive_compute_shortes_df, geometry='geometry', crs=G.graph['crs'])

# 緯度経度 (EPSG:4326) に変換
drive_compute_shortes_df = drive_compute_shortes_df.to_crs(epsg=4326)

# 不要な列を削除する
drive_compute_shortes_df = drive_compute_shortes_df.drop(['path','via'],axis=1)

100%|██████████| 52/52 [00:00<00:00, 18297.30it/s]


In [66]:
all_compute_shortes_df = pd.read_excel('data/compute_shortest_path_all.xlsx')

all_compute_shortes_df = pd.merge(all_compute_shortes_df,all_df,how='left',on=['from_stop_name','to_stop_name'])

# リストに入れるために文字列に変更
all_compute_shortes_df['via'] = all_compute_shortes_df['via'].astype('str')

# via列をリスト形式に変換
all_compute_shortes_df["via"] = all_compute_shortes_df["via"].apply(lambda x: list(map(int, x.split(','))))

# # 最短ルート
all_compute_shortes_df['path'] = all_compute_shortes_df.progress_apply(lambda row: compute_shortest_path(all_G, row['from_osmid'],row['to_osmid'], row['via']), axis=1)

all_compute_shortes_df['geometry'] = all_compute_shortes_df['path'].progress_apply(lambda x: create_geometry(x, all_G))

# # GeoDataFrameを作成し、CRSを設定
all_compute_shortes_df = gpd.GeoDataFrame(all_compute_shortes_df, geometry='geometry', crs=all_G.graph['crs'])

# 緯度経度 (EPSG:4326) に変換
all_compute_shortes_df = all_compute_shortes_df.to_crs(epsg=4326)

# 不要な列を削除する
all_compute_shortes_df = all_compute_shortes_df.drop(['path','via'],axis=1)

100%|██████████| 31/31 [00:00<00:00, 18500.77it/s]


In [81]:
all_gdf = all_gdf[['from_stop_id', 'to_stop_id', 'from_stop_name', 'to_stop_name','from_osmid', 'to_osmid','geometry']]
drive_gdf = drive_gdf[['from_stop_id', 'to_stop_id', 'from_stop_name', 'to_stop_name','from_osmid', 'to_osmid','geometry']]
drive_compute_shortes_gdf = drive_compute_shortes_df[['from_stop_id', 'to_stop_id', 'from_stop_name', 'to_stop_name','from_osmid', 'to_osmid','geometry']]
all_compute_shortes_gdf = all_compute_shortes_df[['from_stop_id', 'to_stop_id', 'from_stop_name', 'to_stop_name','from_osmid', 'to_osmid','geometry']]

In [82]:
import geopandas as gpd
import pandas as pd

# all_gdfをall_compute_shortes_gdfで上書きする
concat_drive_gdf = pd.concat(
    [drive_compute_shortes_gdf, drive_gdf]
).drop_duplicates(
    subset=['from_stop_name', 'to_stop_name'],
    keep='first'
)

# all_gdfをall_compute_shortes_gdfで上書きする
concat_all_gdf = pd.concat(
    [all_compute_shortes_gdf, all_gdf]
).drop_duplicates(
    subset=['from_stop_name', 'to_stop_name'],
    keep='first'
)

In [83]:
# colormap
colormap = cm.linear.RdYlBu_10.scale(0,0.5)
colormap.colors.reverse()

m = folium.Map(location=[34.002755,131.221862], tiles='cartodbpositron', zoom_start=12)

# draw on folium
for idx, row in concat_drive_gdf.iterrows():
    CO2_emission_per_time = 0.3
    color = colormap(CO2_emission_per_time)
    popup_text = f"<b>from_to</b>: {row['from_stop_name']}:{row['to_stop_name']}"
    folium.PolyLine(locations=[(point[1], point[0]) for point in row['geometry'].coords], color='green', weight=2, popup=popup_text).add_to(m)

# draw on folium
for idx, row in concat_all_gdf.iterrows():
    CO2_emission_per_time = 0.3
    color = colormap(CO2_emission_per_time)
    popup_text = f"<b>from_to</b>: {row['from_stop_name']}:{row['to_stop_name']}"
    folium.PolyLine(locations=[(point[1], point[0]) for point in row['geometry'].coords], color='red', weight=2, popup=popup_text).add_to(m)
    
# colormap.caption = 'distance'
m.add_child(colormap)
m